<a href="https://colab.research.google.com/github/goteguru/kmooc_python/blob/main/notebooks/en/kmooc_10_2_sqlite_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Databases

When we talk about data management, we can't ignore the good old relational database systems. Nowadays there are numerous "modern" forms, yet these proven systems still hold a significant portion of our important data.

The classic Python database abstraction is SQLAlchemy, which is an ORM (object-relational mapper) that maps relational databases to objects automatically or semi-automatically.

Since that's quite a "programmer" thing and not very useful unless you want to maintain huge DBMS-independent software systems for years, here I'll present a much simpler option: the Python package for the SQLite engine that works with direct SQL commands.

If you've never learned SQL, it might not be completely clear why it's so useful to be able to create and manage such simple databases, but if you know SQL, you get a very good data management tool basically for free.

SQLite is a super lightweight, serverless database engine that gives us the many advantages of relational databases locally with no installation.

In [ ]:
# As we've come to expect, start with an import
import sqlite3

Because SQLite has no server, an sqlite "connection" essentially means opening a file and giving it a name. With a "real" database you'd provide authentication credentials, server name, port and a schema or database name; none of that is necessary here.  

In [ ]:
# open the "connection"
conn = sqlite3.connect("minta.db")

Through this Connection object we'll be able to manage the database. Of course, just like with files, we should close it at the end! If we don't, we risk losing our changes.

To execute anything we need a database cursor that allows us to run SQL commands. We can have multiple such cursors at the same time.

In [ ]:
cur = conn.cursor()
# and we can already run any SQL.

cur.execute("CREATE TABLE adatok(name TEXT, age INT)")

Done, our table has been created. We could insert data with plain INSERT statements, but due to data validation this is not recommended (it's hard to validate and escape properly), so we'll use placeholders instead.
The data will be placed where the question marks are.

An INSERT (or any modification) automatically starts a transaction in the SQLite engine, so when we've finished the modification we need to close the transaction with a COMMIT as usual in SQL. (Or use ROLLBACK to discard it if something goes wrong).

In [ ]:
data = [
    ("Péter", 23),
    ("Eszter", 19),
    ("Kinga", 33),
]

cur.executemany("INSERT INTO adatok VALUES(?, ?)", data)
conn.commit() # finish the transaction (write out the change)

In [ ]:
# Let's request the data back!
for row in cur.execute("SELECT * FROM adatok ORDER BY name"):
    print(row)

('Eszter', 19)
('Kinga', 33)
('Péter', 23)


In [ ]:
# Finally, (if we're not using it anymore) close our database:
conn.close()

## In-memory database

SQLite doesn't even require a file to operate. It runs happily in memory as well, so if we don't need persistent data we can run it temporarily from memory.

This time let's use a nice context manager instead of manual closing!

In [ ]:
conn = sqlite3.connect(":memory:")

# With the Connection we can also execute SQL; it creates a Cursor for us
cur = conn.execute("CREATE TABLE test(cat, val)")

values = [ # sample data to store
    ("a", 4),
    ("b", 5),
    ("b", 3),
    ("a", 8),
    ("a", 1),
]

with conn: # automatic commit
  cur.executemany("INSERT INTO test VALUES(?, ?)", values)

# and let's query some interesting statistics from it
cur.execute("""
  SELECT cat, sum(val) as sum, avg(val) as avg
  FROM test GROUP BY cat
""")

for cat, sum, avg in cur.fetchall():
  print(cat, sum, avg)

a 13 4.333333333333333
b 8 4.0


## Database exercise

Let's look at a practical example. Suppose we store our data in a directory hierarchy and frequently need to query it. Searching the filesystem can be slow (for example, it might be on a network drive), so we decide to build a database from it to make searching much faster!

For practice, let's read the contents of the /usr/share directory into a database! (There are lots of files there).

In [ ]:
from re import X
import sqlite3
from pathlib import Path

# this will only be in memory for now, but it could be a file...
conn = sqlite3.connect(":memory:")
# create a very simple table for the data
cur = conn.execute("CREATE TABLE files(fn, size)")

with conn:
  for p in Path('/usr/share').glob("**/*"):
    if not p.is_file():
      continue
    size = p.stat().st_size # this is the file size
    cur.execute("INSERT INTO files VALUES (?,?)", (p.name, size))

# and now we can query anything super fast:

In [ ]:
# how many files are longer than 3000 bytes?
cur.execute('SELECT count(*) FROM files WHERE size>3000');
cur.fetchall()

In [ ]:
# largest file size?
cur.execute('SELECT max(size) FROM files');
cur.fetchone()

In [ ]:
# which filename occurs more than 30 times, ordered by count?
cur.execute('SELECT fn, count(*) db FROM files GROUP BY fn HAVING db>30 ORDER BY -db');
cur.fetchall()

That's it! Of course, instead of file size we could have stored other attributes too, like the date or even indexed the file contents (don't do that here right now because there are a lot of files under /usr/share). Instead:

1. Modify the code to also store the path along with the filename, in case someone wants to know exactly where the files are!

2. Create an `input` loop that asks the user for a name fragment and returns all files that contain it (if you haven't learned this, in SQL you can do it like this: `WHERE fn LIKE '%snippet%'`, but feel free to ask an LLM model)